# Logous Model


In [1]:
from Shared.MLModelUtils.Chatbot.ScenarioUtils import ScenarioUtils
from Shared.MLModelUtils.Chatbot.LogousChatbotScenarioBase import LogousChatbotScenarioBase
from Shared.MLModelUtils.Chatbot.BotUtils import BotUtils
import numpy as np
from gensim.models import Word2Vec

class SosoyologoIKScenarioChatbotModel(object):
    
    signatures = None
    def run(self, data, external_files_path = "/"):        
        
        if (SosoyologoIKScenarioChatbotModel.signatures is None):
            import nltk
            nltk.download('punkt')

            import nltk
            nltk.download('stopwords')
            SosoyologoIKScenarioChatbotModel.signatures = np.load(external_files_path + "sample_model_signatures.npy", allow_pickle=True)
            SosoyologoIKScenarioChatbotModel.answers = np.load(external_files_path + "sample_model_answers.npy", allow_pickle=True)
            SosoyologoIKScenarioChatbotModel.questions = np.load(external_files_path + "sample_model_questions.npy", allow_pickle=True)
            SosoyologoIKScenarioChatbotModel.embedding_model = Word2Vec.load(external_files_path+'sample_model_embedding.bin')       
            SosoyologoIKScenarioChatbotModel.botutils = BotUtils(self.get_scenarios(), globals())
            
        result = SosoyologoIKScenarioChatbotModel.botutils.process_msg(data,
                                                                       SosoyologoIKScenarioChatbotModel.signatures, 
                                                                       SosoyologoIKScenarioChatbotModel.answers,
                                                                       SosoyologoIKScenarioChatbotModel.questions, 
                                                                       SosoyologoIKScenarioChatbotModel.embedding_model)
        
        return result
        #from flask import jsonify
        #return jsonify(result)
        
    def get_scenarios(self):
        return [{
                "bot_main_class":"SosyoLogoBot",    
                "entities":{    
                    "startapprove": {            
                    },
                    "gun": {            
                        "msg":"Kaç günlük izin girişi yapmak istiyorsunuz?",              
                    },
                    "start": {            
                        "msg":"Hangi başlangıç tarihi için izin girişi yapmak istiyorsunuz?"            
                    },
                    "tip": {            
                        "msg":"Hangi tipte (yıllık, mazeret, hastalık) izin girmek istiyorsunuz?"            
                    },
                    "finalapprove": {            
                        "msg":"#start başlangıç olmak üzere #tip tipinde #gun günlük izin girişi yapılacak, onaylıyor musunuz?"            
                    }
                },    
                "flow":["startapprove", "gun","start","tip","finalapprove"], 
                "final_msg": "İşleminiz tamamlandı.",      
                "error_msg":"Anlaşılamadı lütfen tekrar giriş yapınız.",
                "abort_msg":"İşleminiz iptal edildi.",
                "ner_model_id" : "1308",
                "ml_api_url" : "http://ml.logo-paas.com:5050/api/run"
                }            
               ]
    
    
class SosyoLogoBot(LogousChatbotScenarioBase):
        
    def ask_entity_startapprove(self, context, sentence, ner_results):        
        msg = ""
        if ("#start" in context["entities"]):
            msg = msg + "#start tarihi için "
        
        if ("#gun" in context["entities"]):
            msg = msg + "#gun gunluk "
            
        msg= msg + "İzin girmek mi istiyorsunuz?"
        return msg, ScenarioUtils.CONTINUE_SCENARIO
        
    def parse_entity_startapprove(self, context, sentence, ner_results):
        result, command = self.parse_yes_no(self, sentence, ner_results)   
        if (command == ScenarioUtils.CONTINUE_SCENARIO) and (result == False):
            return result, ScenarioUtils.ABORT_SCENARIO        
        else:
            return result, command
    
    def parse_entity_gun(self, context, sentence, ner_results):                
        return self.parse_day(self, sentence, ner_results)                
        
    def parse_entity_tip(self, context, sentence, ner_results):
        sentence = sentence.lower().strip()
        if ("yıllık" in sentence) or ("yillik" in sentence):
            return "yıllık", ScenarioUtils.CONTINUE_SCENARIO
        elif ("mazeret" in sentence) or ("mazaret" in sentence):
            return "mazeret", ScenarioUtils.CONTINUE_SCENARIO
        elif ("hasta" in sentence):
            return "hastalik", ScenarioUtils.CONTINUE_SCENARIO    
            
        return None, ScenarioUtils.PARSE_ERROR
            
    def parse_entity_start(self, context, sentence, ner_results):
        return  self.parse_date(self, sentence, ner_results)                    
    
    def parse_entity_finalapprove(self, context, sentence, ner_results):
        result, command = self.parse_yes_no(self, sentence, ner_results)   
        if (command == ScenarioUtils.CONTINUE_SCENARIO) and (result == False):
            return result, ScenarioUtils.ABORT_SCENARIO        
        else:                    
            print("HR services called!")        
            # Todo api call to HR system
            return result, command     

#### Q/A Test

In [2]:
import numpy as np
runner = SosoyologoIKScenarioChatbotModel();
a = runner.run({"question":"iş ilanları"},external_files_path="")
a = a["results"]
a

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'answer': 'Güncel ilanlara şuradan erişebilirsiniz: https://kariyerim.logo.com.tr/RecruitPortal/#!announcement'}

#### Scenario Test

In [3]:
# Test Production Ready Model
runner = SosoyologoIKScenarioChatbotModel();
c1 = runner.run({"question":"izin istiyorum"},external_files_path="")
c1 = c1["results"]
c1

{'answer': 'İzin girmek mi istiyorsunuz?',
 'context': {'current_entity': 'startapprove',
  'entities': {},
  'scenario_id': 0}}

In [4]:
from copy import deepcopy
c2 = runner.run({"question":"evet izin girmek istiyorum", "context":deepcopy(c1["context"])},external_files_path="")
c2 = c2["results"]
c2

Kaç günlük izin girişi yapmak istiyorsunuz?


{'answer': 'Kaç günlük izin girişi yapmak istiyorsunuz?',
 'context': {'current_entity': 'gun',
  'entities': {'startapprove': True},
  'scenario_id': 0}}

In [5]:
from copy import deepcopy
c3 = runner.run({"question":"2 haftalık", "context":deepcopy(c2["context"])},external_files_path="")
c3 = c3["results"]
c3

Hangi başlangıç tarihi için izin girişi yapmak istiyorsunuz?


{'answer': 'Hangi başlangıç tarihi için izin girişi yapmak istiyorsunuz?',
 'context': {'current_entity': 'start',
  'entities': {'gun': 14, 'startapprove': True},
  'scenario_id': 0}}

In [6]:
from copy import deepcopy
c4 = runner.run({"question":"iptal", "context":deepcopy(c3["context"])},external_files_path="")
c4 = c4["results"]
c4

İşleminiz iptal edildi.


{'answer': 'İşleminiz iptal edildi.', 'context': None}